In [1]:
from core.thresholds import PARAM_THRESHOLDS

In [2]:
import asyncio
from datetime import datetime, timedelta
from typing import List, Any, Dict

import aiohttp
import pandas as pd
import numpy as np

async def fetch_monthly_data_from_power_dav(
    session: aiohttp.ClientSession,
    latitude: float,
    longitude: float,
    start_year: str,
    end_year: str,
    parameters: List[str]
) -> Dict[str, Any]:
    url = (
        "https://power.larc.nasa.gov/api/temporal/monthly/point"
        f"?start={start_year}&end={end_year}&latitude={latitude}&longitude={longitude}"
        f"&parameters={','.join(parameters)}&format=JSON&community=RE"
    )
    async with session.get(url) as resp:
        resp.raise_for_status()
        js = await resp.json()
        return js["properties"]["parameter"]

async def get_monthly_data_async(target_date: datetime, latitude, longitude, params, window=0, years_back=5):
    dfs = []
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i in range(1, years_back+1):
            past_year = target_date.year - i
            start = (target_date.replace(year=past_year) - timedelta(days=window)).strftime("%Y")
            end = (target_date.replace(year=past_year) + timedelta(days=window)).strftime("%Y")
            tasks.append(fetch_monthly_data_from_power_dav(session, latitude, longitude, start, end, params))
        results = await asyncio.gather(*tasks)

    # convert to dataframe list
    for i, r in enumerate(results, 1):
        past_year = target_date.year - i
        for p in params:
            del r[p][f"{past_year}13"]
        df = pd.DataFrame({p: pd.Series(r[p]) for p in params})
        df.index = pd.to_datetime(df.index, format="%Y%m")
        df["year"] = past_year
        df["month"] = df.index.month
        dfs.append(df)

    full_df = pd.concat(dfs, axis=0)
    return full_df.reset_index(drop=True)

# Create monthly averages
def make_hourly_yearly_avg_df(raw_df, params):
    avg_df = raw_df.groupby(["month"])[params].mean().reset_index()
    return avg_df

# Example usage
params = ["T2M","RH2M","PRECTOTCORR","ALLSKY_SFC_SW_DWN","WS2M"]
target_date = datetime(2025,10,2,12)

raw_df = await get_monthly_data_async(target_date, 10.75, 106.67, params)
avg_df = make_hourly_yearly_avg_df(raw_df, params)

In [10]:
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from datetime import datetime

from typing import List, Dict

def prepare_plotly_fanmap_with_vbands_v3(
    pred_df: pd.DataFrame, parameter, ci_levels=[30, 60, 90]
):
    sns.set_style("whitegrid")
    x = pred_df["month"].tolist()
    x_dt = pred_df["month"]  # giữ dạng datetime
    traces = []


    # Mean line
    mean_vals = pred_df[parameter].tolist()
    traces.append(
        go.Scatter(
            x=x,
            y=mean_vals,
            mode="lines+markers",
            line=dict(color="black", width=2),
            name="Mean",
            hovertemplate=f"Time: %{{x}}<br>{parameter}: %{{y:.2f}}<extra></extra>",
        )
    )

    fig = go.Figure(data=traces)

    # --- Beaufort classes ---
    thresholds = PARAM_THRESHOLDS[parameter]

    def classify(val):
        for threshold in thresholds:
            if threshold["lower"] <= val <= threshold["upper"]:
                return threshold
        return None

    # Gom các đoạn liên tiếp cùng loại gió
    bands = []
    current_class = None
    x0 = None

    for i, val in enumerate(mean_vals):
        cls = classify(val)
        if cls != current_class:
            if current_class is not None:
                bands.append((x0, x_dt[i], current_class))
            current_class = cls
            x0 = x_dt[i]
    if current_class is not None:
        bands.append((x0, x_dt.iloc[-1], current_class))

    # Vẽ các vrect gom nhóm theo Beaufort
    for x0, x1, wc in bands:
        if x0 == x1:
            x1 += 0.05
        fig.add_vrect(
            x0=x0,
            x1=x1,
            fillcolor=wc["color"],
            opacity=0.3,  # tăng độ đậm
            layer="below",
            line_width=0,
            annotation_text=wc["label"],
            annotation_position="top left",
            annotation=dict(font_size=12, font_color="black", font_weight="bold"),
        )


    # Thêm legend (trace ẩn cho mỗi loại gió)
    for wc in thresholds:
        fig.add_trace(
            go.Scatter(
                x=[None],
                y=[None],
                mode="markers",
                marker=dict(size=10, color=wc["color"]),
                name=wc["label"],
            )
        )

    # --- Local maxima / minima ---
    y = np.array(mean_vals)
    maxima = np.r_[False, (y[1:-1] > y[:-2]) & (y[1:-1] > y[2:]), False]
    minima = np.r_[False, (y[1:-1] < y[:-2]) & (y[1:-1] < y[2:]), False]

    fig.add_trace(
        go.Scatter(
            x=np.array(x)[maxima],
            y=y[maxima],
            mode="markers",
            marker=dict(color="red", size=10, symbol="circle"),
            name="Max",
            hovertemplate=f"Time: %{{x}}<br>{parameter}: %{{y:.2f}}<extra></extra>",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=np.array(x)[minima],
            y=y[minima],
            mode="markers",
            marker=dict(color="blue", size=10, symbol="circle"),
            name="Min",
            hovertemplate=f"Time: %{{x}}<br>{parameter}: %{{y:.2f}}<extra></extra>",
        )
    )

    # Layout
    fig.update_layout(
        title=f"Fan Map for {parameter} on {datetime.now().year}",
        xaxis_title="Time",
        yaxis_title=parameter,
        template="simple_white",
    )

    # Hiển thị lưới
    fig.update_xaxes(tickmode="array", tickvals=list(range(1, 13)), ticktext=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"])
    fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor="lightgray")


    # Ví dụ tháng hiện tại
    current_month = datetime.now().month   # hoặc gán = 10 nếu muốn fix

    fig.add_vline(
        x=current_month,  # vị trí đường kẻ (theo trục X)
        line_width=2,
        line_dash="dash",  # kiểu nét đứt
        line_color="darkred",
        annotation_text=f"Current month",  # label trên đường
        annotation_position="top",
        annotation=dict(font_size=12, font_color="black", font_weight="bold", font_style="normal"),
    )

    fig.show()

    # return fig

prepare_plotly_fanmap_with_vbands_v3(avg_df, "PRECTOTCORR")

ValueError: Invalid property specified for object of type plotly.graph_objs.layout.annotation.Font: 'face'

Did you mean "size"?

    Valid properties:
        color

        family
            HTML font family - the typeface that will be applied by
            the web browser. The web browser can only apply a font
            if it is available on the system where it runs. Provide
            multiple font families, separated by commas, to
            indicate the order in which to apply fonts if they
            aren't available.
        lineposition
            Sets the kind of decoration line(s) with text, such as
            an "under", "over" or "through" as well as combinations
            e.g. "under+over", etc.
        shadow
            Sets the shape and color of the shadow behind text.
            "auto" places minimal shadow and applies contrast text
            font color. See https://developer.mozilla.org/en-
            US/docs/Web/CSS/text-shadow for additional options